# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [9]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from urllib.request import urlretrieve
import gzip

# Manual download URLs
TRAIN_IMAGES_URL = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz'
TRAIN_LABELS_URL = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz'
TEST_IMAGES_URL = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz'
TEST_LABELS_URL = 'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz'

def download_fashion_mnist(data_dir='./data'):
    os.makedirs(data_dir, exist_ok=True)
    for url in [TRAIN_IMAGES_URL, TRAIN_LABELS_URL, TEST_IMAGES_URL, TEST_LABELS_URL]:
        filename = os.path.join(data_dir, url.split('/')[-1])
        if not os.path.exists(filename):
            print(f"Downloading {filename}...")
            urlretrieve(url, filename)

def load_fashion_mnist(data_dir='./data', train=True):
    if train:
        images_path = os.path.join(data_dir, 'train-images-idx3-ubyte.gz')
        labels_path = os.path.join(data_dir, 'train-labels-idx1-ubyte.gz')
    else:
        images_path = os.path.join(data_dir, 't10k-images-idx3-ubyte.gz')
        labels_path = os.path.join(data_dir, 't10k-labels-idx1-ubyte.gz')
    
    # Load images
    with gzip.open(images_path, 'rb') as f:
        images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)
    
    # Load labels
    with gzip.open(labels_path, 'rb') as f:
        labels = np.frombuffer(f.read(), np.uint8, offset=8)
    
    return images, labels

class CustomFashionMNIST(Dataset):
    def __init__(self, root='./data', train=True, transform=None):
        download_fashion_mnist(root)
        self.data, self.targets = load_fashion_mnist(root, train)
        self.data = self.data.astype(np.float32) / 255.0
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        image = torch.FloatTensor(image).unsqueeze(0)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [10]:
class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def save_model(model, optimizer, epoch, accuracy, is_best=False):
    """Save model weights - both best and last"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': accuracy
    }
# Always save the last model
    last_filename = 'fashion_model_last.pt'
    torch.save(checkpoint, last_filename)  # Removed weights_only parameter
    print(f"Last model saved to {last_filename}")
    
    # Save best model separately if it's the best
    if is_best:
        best_filename = 'fashion_model_best.pt'
        torch.save(checkpoint, best_filename)  # Removed weights_only parameter
        print(f"Best model saved to {best_filename}")

def load_model(filename='fashion_model_best.pt'):
    """Load model weights"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = FashionNet().to(device)
    
    try:
        checkpoint = torch.load(filename, map_location=device)  # Removed weights_only parameter
        model.load_state_dict(checkpoint['model_state_dict'])
        accuracy = checkpoint.get('accuracy', 0.0)
        epoch = checkpoint.get('epoch', 0)
        print(f"Model loaded from {filename}")
        print(f"Accuracy: {accuracy:.2f}%")
        return model, accuracy, epoch
    except FileNotFoundError:
        print(f"No saved model found at {filename}")
        return model, 0.0, 0
if __name__ == "__main__":
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create datasets and loaders
    train_dataset = CustomFashionMNIST(train=True)
    test_dataset = CustomFashionMNIST(train=False)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    # Create model
    model = FashionNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Train for 10 epochs
    num_epochs = 10
    best_accuracy = 0.0
    
    print("Starting training...")
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')
        
        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch: {epoch}, Accuracy: {accuracy:.2f}%')
        
        # Save both best and last models
        is_best = accuracy > best_accuracy
        if is_best:
            best_accuracy = accuracy
            print(f'New best accuracy: {accuracy:.2f}%')
        save_model(model, optimizer, epoch, accuracy, is_best=is_best)
        
    print(f"\nTraining completed!")
    print(f"Best accuracy achieved: {best_accuracy:.2f}%")

Using device: cpu
Starting training...
Epoch: 0, Accuracy: 90.04%
New best accuracy: 90.04%
Last model saved to fashion_model_last.pt
Best model saved to fashion_model_best.pt
Epoch: 1, Accuracy: 90.64%
New best accuracy: 90.64%
Last model saved to fashion_model_last.pt
Best model saved to fashion_model_best.pt
Epoch: 2, Accuracy: 91.59%
New best accuracy: 91.59%
Last model saved to fashion_model_last.pt
Best model saved to fashion_model_best.pt
Epoch: 3, Accuracy: 90.44%
Last model saved to fashion_model_last.pt
Epoch: 4, Accuracy: 91.95%
New best accuracy: 91.95%
Last model saved to fashion_model_last.pt
Best model saved to fashion_model_best.pt
Epoch: 5, Accuracy: 91.97%
New best accuracy: 91.97%
Last model saved to fashion_model_last.pt
Best model saved to fashion_model_best.pt
Epoch: 6, Accuracy: 92.47%
New best accuracy: 92.47%
Last model saved to fashion_model_last.pt
Best model saved to fashion_model_best.pt
Epoch: 7, Accuracy: 92.39%
Last model saved to fashion_model_last.pt
E

In [11]:
import torch
from torch.utils.data import DataLoader


def evaluate_model(model_path='fashion_model_best.pt'):  # Fixed function definition
    """Evaluate a saved model"""
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    try:
        # Load the saved model
        model, saved_accuracy, last_epoch = load_model(model_path)  # Now model_path is defined
        model.eval()

        # Create test dataset
        test_dataset = CustomFashionMNIST(train=False)
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Evaluation
        correct = 0
        total = 0

        print(f"\nEvaluating model from epoch {last_epoch}")
        print(f"Saved accuracy: {saved_accuracy:.2f}%")

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        current_accuracy = 100 * correct / total
        print(f'Current Evaluation Accuracy: {current_accuracy:.2f}%')
        # Show sample predictions
        print("Sample Predictions:")
        num_samples = 5  # Number of sample predictions to show
        for i in range(num_samples):
            sample_image, sample_label = test_dataset[i]
            sample_image = sample_image.unsqueeze(0).to(device)
            prediction = model(sample_image).argmax(1)[0].item()
            print(f"Sample {i + 1}:")
            print(f"Model predicts: {test_dataset.classes[prediction]}")
            print(f"Actual label: {test_dataset.classes[sample_label]}")
            print()
            
    except FileNotFoundError:
        print(f"Error: Could not find model file '{model_path}'")
    except Exception as e:
        print(f"Error during evaluation: {str(e)}")

if __name__ == "__main__":    # Fixed the name dunder
    print(" Fashion MNIST Model Evaluation")
    
    print("Evaluating Best Model:")
    evaluate_model('fashion_model_best.pt')
    
    print("Evaluating Last Model:")
    evaluate_model('fashion_model_last.pt')

 Fashion MNIST Model Evaluation
Evaluating Best Model:
Using device: cpu
Model loaded from fashion_model_best.pt
Accuracy: 92.84%

Evaluating model from epoch 9
Saved accuracy: 92.84%
Current Evaluation Accuracy: 92.84%
Sample Predictions:
Sample 1:
Error during evaluation: 'CustomFashionMNIST' object has no attribute 'classes'
Evaluating Last Model:
Using device: cpu
Model loaded from fashion_model_last.pt
Accuracy: 92.84%

Evaluating model from epoch 9
Saved accuracy: 92.84%
Current Evaluation Accuracy: 92.84%
Sample Predictions:
Sample 1:
Error during evaluation: 'CustomFashionMNIST' object has no attribute 'classes'
